### Calculate Top-k Accuracy (TTA20)

In [1]:
from rdkit import Chem
from neutral import NeutraliseCharges
from multiprocessing import Pool
from rdkit import RDLogger
import pandas as pd
from rdkit.Chem.MolStandardize import rdMolStandardize
RDLogger.DisableLog('rdApp.*')

def canonicalize_smiles(smiles):
    if len(smiles)==0:
        return ''
    mol = Chem.MolFromSmiles(smiles)
    # lfc = MolStandardize.fragment.LargestFragmentChooser()
    
    if mol is not None:
        # mol2 = lfc.choose(mol)
        smi2=Chem.MolToSmiles(mol, isomericSmiles=True)
        smi,_=NeutraliseCharges(smi2)
        return smi 
    else:
        return ''
# df['substrate']=df['substrate'].apply(canonicalize_smiles)

In [2]:
smi="S C [ C @ H ] ( N ) C ( = O ) N [ C @ @ H ] ( C S ) C ( = O ) N [ C @ @ H ] ( C C ( N ) = O ) C ( = O ) N 1 [ C @ H ] ( C ( N [ C @ @ H ] ( C ) C ( = O ) O ) = O ) C C C 1 . O = C ( O ) [ C @ H ] ( [ C @ @ H ] ( C ) O ) N C ( [ C @ @ H ] ( N ) C S ) = O"
smi.replace(" ","")

'SC[C@H](N)C(=O)N[C@@H](CS)C(=O)N[C@@H](CC(N)=O)C(=O)N1[C@H](C(N[C@@H](C)C(=O)O)=O)CCC1.O=C(O)[C@H]([C@@H](C)O)NC([C@@H](N)CS)=O'

In [131]:
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem import MolStandardize
def canonicalize_predict(smiles):
    if len(smiles)==0:
        return ''
    mol = Chem.MolFromSmiles(smiles)
    lfc = MolStandardize.fragment.LargestFragmentChooser()
    
    if mol is not None:
        mol2 = lfc.choose(mol)
        smi2=Chem.MolToSmiles(mol2, isomericSmiles=True)
        smi,_=NeutraliseCharges(smi2)
        return smi
    else:
        return ''

In [132]:
# sub=list(map(remove_leavegroup,true_reactseq_lis))

## label

In [133]:
import os
import pandas as pd
from e_smiles import get_e_smiles, merge_smiles, get_edit_from_e_smiles
from joblib import Parallel, delayed
from rdkit import Chem
from rdkit.DataStructs import TanimotoSimilarity
from rdkit.Chem import AllChem

def process_smiles(smiles):
    return merge_smiles(smiles)




In [134]:
def kekulize_smiles(smi):
    smi = smi.replace(' ','')
    mol = Chem.MolFromSmiles(smi)
    Chem.Kekulize(mol, clearAromaticFlags = True)
    smi = Chem.MolToSmiles(mol, canonical = False, kekuleSmiles = True)
    return smi

In [135]:
n_best = 10
substrate_lis = []
true_tgt_lis = []
with open('test_enzyme_argue10_input_group.txt') as f:
    for line in f.readlines():
        line = line.replace('\n','').replace(" ","")
        substrate_lis += [line] * n_best
        true_tgt_lis.append(line)

In [136]:
len(true_tgt_lis)

510

In [137]:
import csv

# 输入文件路径
input_file_path = 'test_enzyme_nbest10_argue10.txt'
# 输出文件路径，使用.tsv后缀
output_file_path = 'test_enzyme_argue10.tsv'

# 打开输出的TSV文件
with open(output_file_path, 'w', newline='') as tsvfile:
    # 定义列名
    fieldnames = ['enzyme', 'metabolite']
    # 创建CSV写入器，使用\t作为分隔符
    writer = csv.DictWriter(tsvfile, fieldnames=fieldnames, delimiter='\t')
    
    # 写入列名
    writer.writeheader()

    # 打开并读取输入文件
    with open(input_file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if '|' in line:
                # 分割酶名称和化学结构
                enzyme, metabolite = line.split('|')
                enzyme = enzyme.strip()
                metabolite = metabolite.replace(' ', '').strip()  # 去掉metabolite中的所有空格
            else:
                # 如果没有 '|'，酶的名称设为 None，metabolite 保持不变
                enzyme = None
                metabolite = line.replace(' ', '').strip()  # 也去掉空格

            # 写入一行到TSV文件
            writer.writerow({'enzyme': enzyme, 'metabolite': metabolite})

print(f"Data has been written to {output_file_path}")


Data has been written to test_enzyme_argue10.tsv


In [138]:
import csv

pred_tgt_lis = []
with open('test_enzyme_argue10.tsv', 'r') as f:
    reader = csv.DictReader(f, delimiter='\t')
    for row in reader:
        metabolite = row['metabolite'].replace(" ", "")  # 去除空格
        pred_tgt_lis.append(metabolite)

# 打印结果以确认
print(pred_tgt_lis)


['CC1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2[~OH]<O=C(O)C1O[CH:1][C@H](O)[C@@H](O)[C@@H]1O>', 'CC1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2;O', '[~CH3]C1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2O<[OH:1]>', 'CC1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2[~OH]<O=C(O)C1O[CH:1][C@H](O)[C@@H](O)[C@@H]1O>', 'CC1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2;O', 'CC1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2;O', 'CC1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2;O', 'CC1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2[~OH]<O=C(O)C1O[CH:1][C@H](O)[C@@H](O)[C@@H]1O>', 'CC1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2[~OH]<O=C(O)C1O[CH:1][C@H](O)[C@@H](O)[C@@H]1O>', 'CC1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2[~OH]', 'C1=C(Cl)C=CC2=C1C(C1=C(F)C=CC=C1)=NC(O)C1=CN=C(!C)N21<[OH:1]><[Au:1]>', 'C1=C(Cl)C=CC2=C1C(C1=C(F)C=CC=C1)=NC(O)C1=CN=C(!C)N21<><[Au:1]>', 'C1=C(Cl)C=CC2=C1C(C1=C(F)C=CC=C1)=NC(O)C1=CN=C(!C)N21<[OH:1]><[Au:1]>', 'C1=C(Cl)C=CC2=C1C(C1=C(F)C=CC=C1)=NC(O)C1=[~CH]N=C(C)N21<[OH:1]>', 'C1=C(Cl)C=CC

In [139]:
pred = [f"{x}>>>{y}" for x, y in zip(substrate_lis, pred_tgt_lis)]
pred_smi= Parallel(n_jobs=-1)(delayed(process_smiles)(x) for x in pred)

rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2rdkit version:2019.03.2

rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:2019.03.2
rdkit version:20

In [140]:
len(pred_smi)

5100

In [141]:
def remove_leavegroup(smi):
    frag = smi.split(".")
    result = [i for i in frag if 'Au' not in i]
    return ".".join(result)

predict_smiles_all=list(map(remove_leavegroup,pred_smi))


In [142]:
predict_cocan_smiles=list(map(canonicalize_smiles,predict_smiles_all))

In [143]:
len(predict_cocan_smiles)

5100

In [144]:
len(predict_cocan_smiles)

5100

In [145]:
import pandas as pd
data=pd.read_csv("test_enzyme_argue10.tsv",sep='\t')

In [146]:
data

,enzyme,metabolite
0,UGT,CC1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2[...
1,CYP3A4,CC1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2;O
2,CYP3A4,[~CH3]C1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)...
3,UGT1A9,CC1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2[...
4,CYP2C9,CC1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2;O
...,...,...
5095,UGT2B7,[?C@@H]1([~OH])[C@H](C)[?C@H]2[?C@]34OO[?C@](C...
5096,UGT2B7,[?C@@H]1([~OH])[C@H](C)[?C@H]2[C@]34OO[?C@](C)...
5097,UGT2B7,[C@@H]1([~OH])[C@H](C)[?C@H]2[C@]34OO[?C@](C)(...
5098,UGT2B7,[?C@@H]1([~OH])[C@H](C)[C@H]2[C@]34OO[C@](C)(O...


In [147]:
data['smiles']=predict_cocan_smiles

In [148]:
data.to_csv("test_enzyme_argue10_add_smiles.tsv",index=False)

In [149]:
data=pd.read_csv("test_enzyme_argue10_add_smiles.tsv")

In [150]:
data

,enzyme,metabolite,smiles
0,UGT,CC1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2[...,Cc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc1F)=NC2OC1OC(C(...
1,CYP3A4,CC1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2;O,Cc1ncc2c(=O)nc(-c3ccccc3F)c3cc(Cl)ccc3n12
2,CYP3A4,[~CH3]C1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)...,OCc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc1F)=NC2O
3,UGT1A9,CC1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2[...,Cc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc1F)=NC2OC1OC(C(...
4,CYP2C9,CC1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2;O,Cc1ncc2c(=O)nc(-c3ccccc3F)c3cc(Cl)ccc3n12
...,...,...,...
5095,UGT2B7,[?C@@H]1([~OH])[C@H](C)[?C@H]2[?C@]34OO[?C@](C...,CC1CCC2[C@@H](C)C(OC3OC(C(=O)O)C(O)C(O)C3O)OC3...
5096,UGT2B7,[?C@@H]1([~OH])[C@H](C)[?C@H]2[C@]34OO[?C@](C)...,CC1CCC2[C@@H](C)C(OC3OC(C(=O)O)C(O)C(O)C3O)OC3...
5097,UGT2B7,[C@@H]1([~OH])[C@H](C)[?C@H]2[C@]34OO[?C@](C)(...,CC1CCC2[C@@H](C)[C@@H](OC3OC(C(=O)O)C(O)C(O)C3...
5098,UGT2B7,[?C@@H]1([~OH])[C@H](C)[C@H]2[C@]34OO[C@](C)(O...,C[C@@H]1CC[C@H]2[C@@H](C)C(OC3OC(C(=O)O)[C@H](...


In [151]:
n_best=10

In [152]:
enzymes = data['enzyme'].fillna('').astype(str).tolist()
predict_cocan_smiles = data['smiles'].fillna('').tolist()

fold = 10
num = 51
n = n_best  
enzyme_prefixes = ['CYP', 'UGT', 'SULT', 'AKR', 'GST', 'CES', 'FMO', 'ALDH', 'NAT', 'COMT', 'AOX', 'XDH', 'EPHX', 'NQO', 'ADH']

# 计算分数
score_1 = []
for i in range(fold):
    for j in range(num):
        for k in range(1, n + 1):
            score = 1 / k**2
            score_1.append(score)

# 初始化
vote_score_lis = [-1] * len(enzymes)
id_list = []

# 计算 vote_score_lis
for j in range(num):
    enzyme_score_dic = {}
    enzyme_category_map = {}

    for i in range(fold):
        for k in range(n):
            idx = j * fold * n + i * n + k
            id_list.append(idx)
            
            enzyme = enzymes[idx]
            s = score_1[idx] if enzyme != '' else 0
            
            if enzyme == '':
                continue  # 跳过空值

            # 根据 enzyme 的前缀分类
            enzyme_category = None
            for prefix in enzyme_prefixes:
                if enzyme.startswith(prefix):
                    enzyme_category = prefix
                    break

            if enzyme_category is None:
                continue  # 如果没有匹配到大类，跳过
            
            # 大类和亚型分别累积打分
            key = enzyme_category
            subtype_key = enzyme

            # 大类累积打分
            if key not in enzyme_score_dic:
                enzyme_score_dic[key] = s
            else:
                enzyme_score_dic[key] += s

            # 亚型累积打分
            if subtype_key not in enzyme_category_map:
                enzyme_category_map[subtype_key] = s
            else:
                enzyme_category_map[subtype_key] += s

    # 更新 vote_score_lis
    for i in range(fold):
        for k in range(n):
            idx = j * fold * n + i * n + k
            enzyme = enzymes[idx]

            # 根据大类和亚型分开处理
            enzyme_category = None
            for prefix in enzyme_prefixes:
                if enzyme.startswith(prefix):
                    enzyme_category = prefix
                    break

            if enzyme_category is None:
                continue  # 如果没有匹配到大类，跳过

            # 更新总分
            vote_score_lis[idx] = enzyme_category_map.get(enzyme, -1)

# 创建最终的 pre_enzyme_category_50_lis 和 pre_enzyme_50_lis
pre_enzyme_category_50_lis = []
pre_enzyme_50_lis = []

for j in range(num):
    enzyme_g = []
    vote_score_g = []
    
    for i in range(fold):
        for k in range(n):
            idx = j * fold * n + i * n + k
            enzyme_g.append(enzymes[idx])
            vote_score_g.append(vote_score_lis[idx])
            
    zip_a_b = zip(enzyme_g, vote_score_g)
    sorted_zip = sorted(zip_a_b, key=lambda x: x[1], reverse=True)
    enzyme_g, vote_score_g = zip(*sorted_zip)
    
    pre_enzyme_category_50 = []
    pre_enzyme_50 = []
    
    seen_enzyme_category = set()
    seen_enzyme = set()
    
    for enzyme in enzyme_g:
        enzyme_category = None
        for prefix in enzyme_prefixes:
            if enzyme.startswith(prefix):
                enzyme_category = prefix
                break

        if enzyme_category is None:
            continue  # 如果没有匹配到大类，跳过

        # 先检查是否已经有该大类的条目
        if enzyme_category in seen_enzyme_category:
            # 如果大类已经存在，则仅保留带有亚型的酶
            if enzyme != enzyme_category and enzyme not in seen_enzyme and len(pre_enzyme_50) < 50:
                pre_enzyme_50.append(enzyme)
                seen_enzyme.add(enzyme)
        else:
            # 大类处理
            if len(pre_enzyme_category_50) < 50:
                pre_enzyme_category_50.append(enzyme_category)
                seen_enzyme_category.add(enzyme_category)

            # 亚型处理
            if enzyme != enzyme_category and len(pre_enzyme_50) < 50:
                pre_enzyme_50.append(enzyme)
                seen_enzyme.add(enzyme)
    
    pre_enzyme_category_50.extend([''] * (50 - len(pre_enzyme_category_50)))
    pre_enzyme_50.extend([''] * (50 - len(pre_enzyme_50)))

    pre_enzyme_category_50_lis.append(pre_enzyme_category_50)
    pre_enzyme_50_lis.append(pre_enzyme_50)


In [153]:
# 初始化最终的列表
combined_list = []

# 合并 pre_enzyme_category_50_lis 和 pre_enzyme_50_lis，只保留非空元素
for category_list, enzyme_list in zip(pre_enzyme_category_50_lis, pre_enzyme_50_lis):
    # 过滤掉空字符串
    filtered_category = [item for item in category_list if item != '']
    filtered_enzyme = [item for item in enzyme_list if item != '']
    
    # 合并非空元素
    combined = filtered_category + filtered_enzyme
    combined_list.append(combined)

# 查看合并后的列表
for combined in combined_list:
    print(combined)


['CYP', 'UGT', 'CYP3A4', 'CYP2C9', 'CYP2C19', 'CYP1A2', 'CYP2D6', 'CYP3A5', 'CYP1A1', 'CYP2B6', 'UGT1A9', 'UGT1A1', 'CYP2C8', 'UGT1A3']
['CYP', 'UGT', 'CYP3A4', 'UGT1A9', 'CYP2C8', 'UGT2B7', 'UGT1A1', 'UGT1A8', 'UGT2B4', 'CYP2D6', 'CYP2C19', 'CYP1A1']
['CYP', 'CYP2C19', 'CYP2D6', 'CYP3A4', 'CYP1A2', 'CYP2C9', 'CYP2C8', 'CYP1A1']
['CYP', 'UGT', 'CYP3A4', 'CYP2D6', 'CYP1A2', 'CYP1A1', 'CYP2C19', 'CYP2C8', 'CYP2C9', 'CYP3A5', 'UGT1A1']
['CYP', 'CYP3A4', 'CYP2D6', 'CYP1A1', 'CYP1A2']
['CYP', 'CYP2D6', 'CYP3A4', 'CYP1A2', 'CYP1A1', 'CYP2C8', 'CYP2C19', 'CYP2A6', 'CYP2C9', 'CYP2B6', 'CYP1A', 'CYP3A5']
['CYP', 'UGT', 'CYP3A4', 'UGT1A1', 'CYP1A2', 'UGT1A9', 'CYP2C19', 'UGT1A3', 'CYP2C9', 'UGT1A10', 'CYP2B6', 'CYP2D6', 'CYP1A1', 'CYP2A6', 'CYP3A5']
['CYP', 'CYP3A4', 'CYP1A1', 'CYP1A2', 'CYP2D6', 'CYP2C8', 'CYP2C19', 'CYP3A5', 'CYP2C9', 'CYP1B1']
['CYP', 'CYP3A4', 'CYP1A1', 'CYP1A2', 'CYP3A5', 'CYP2D6', 'CYP2C9', 'CYP2C19']
['CYP', 'CYP2C19', 'CYP1A2', 'CYP2B6', 'CYP3A4', 'CYP2C8', 'CYP1A1', 'CY

In [154]:
combined_list

[['CYP',
  'UGT',
  'CYP3A4',
  'CYP2C9',
  'CYP2C19',
  'CYP1A2',
  'CYP2D6',
  'CYP3A5',
  'CYP1A1',
  'CYP2B6',
  'UGT1A9',
  'UGT1A1',
  'CYP2C8',
  'UGT1A3'],
 ['CYP',
  'UGT',
  'CYP3A4',
  'UGT1A9',
  'CYP2C8',
  'UGT2B7',
  'UGT1A1',
  'UGT1A8',
  'UGT2B4',
  'CYP2D6',
  'CYP2C19',
  'CYP1A1'],
 ['CYP',
  'CYP2C19',
  'CYP2D6',
  'CYP3A4',
  'CYP1A2',
  'CYP2C9',
  'CYP2C8',
  'CYP1A1'],
 ['CYP',
  'UGT',
  'CYP3A4',
  'CYP2D6',
  'CYP1A2',
  'CYP1A1',
  'CYP2C19',
  'CYP2C8',
  'CYP2C9',
  'CYP3A5',
  'UGT1A1'],
 ['CYP', 'CYP3A4', 'CYP2D6', 'CYP1A1', 'CYP1A2'],
 ['CYP',
  'CYP2D6',
  'CYP3A4',
  'CYP1A2',
  'CYP1A1',
  'CYP2C8',
  'CYP2C19',
  'CYP2A6',
  'CYP2C9',
  'CYP2B6',
  'CYP1A',
  'CYP3A5'],
 ['CYP',
  'UGT',
  'CYP3A4',
  'UGT1A1',
  'CYP1A2',
  'UGT1A9',
  'CYP2C19',
  'UGT1A3',
  'CYP2C9',
  'UGT1A10',
  'CYP2B6',
  'CYP2D6',
  'CYP1A1',
  'CYP2A6',
  'CYP3A5'],
 ['CYP',
  'CYP3A4',
  'CYP1A1',
  'CYP1A2',
  'CYP2D6',
  'CYP2C8',
  'CYP2C19',
  'CYP3A5',
  'CYP2C9'

In [155]:
len(pre_enzyme_category_50_lis)

51

In [156]:
pre_enzyme_category_50_lis[0]

['CYP',
 'UGT',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [157]:
pre_enzyme_50_lis[0]

['CYP3A4',
 'CYP2C9',
 'CYP2C19',
 'CYP1A2',
 'CYP2D6',
 'CYP3A5',
 'CYP1A1',
 'CYP2B6',
 'UGT1A9',
 'UGT1A1',
 'CYP2C8',
 'UGT1A3',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [158]:
num

51

In [159]:
len(combined_list)

51

In [160]:
for i in range(len(combined_list)):
    combined_list[i] = combined_list[i] + [""] * (20 - len(combined_list[i]))

In [161]:
# num=65
all=[]
for i in range(num):
    data=[]
    for k in range(20):     
        data.append(combined_list[i][k])
    all.append("|".join(data))

In [162]:
all

['CYP|UGT|CYP3A4|CYP2C9|CYP2C19|CYP1A2|CYP2D6|CYP3A5|CYP1A1|CYP2B6|UGT1A9|UGT1A1|CYP2C8|UGT1A3||||||',
 'CYP|UGT|CYP3A4|UGT1A9|CYP2C8|UGT2B7|UGT1A1|UGT1A8|UGT2B4|CYP2D6|CYP2C19|CYP1A1||||||||',
 'CYP|CYP2C19|CYP2D6|CYP3A4|CYP1A2|CYP2C9|CYP2C8|CYP1A1||||||||||||',
 'CYP|UGT|CYP3A4|CYP2D6|CYP1A2|CYP1A1|CYP2C19|CYP2C8|CYP2C9|CYP3A5|UGT1A1|||||||||',
 'CYP|CYP3A4|CYP2D6|CYP1A1|CYP1A2|||||||||||||||',
 'CYP|CYP2D6|CYP3A4|CYP1A2|CYP1A1|CYP2C8|CYP2C19|CYP2A6|CYP2C9|CYP2B6|CYP1A|CYP3A5||||||||',
 'CYP|UGT|CYP3A4|UGT1A1|CYP1A2|UGT1A9|CYP2C19|UGT1A3|CYP2C9|UGT1A10|CYP2B6|CYP2D6|CYP1A1|CYP2A6|CYP3A5|||||',
 'CYP|CYP3A4|CYP1A1|CYP1A2|CYP2D6|CYP2C8|CYP2C19|CYP3A5|CYP2C9|CYP1B1||||||||||',
 'CYP|CYP3A4|CYP1A1|CYP1A2|CYP3A5|CYP2D6|CYP2C9|CYP2C19||||||||||||',
 'CYP|CYP2C19|CYP1A2|CYP2B6|CYP3A4|CYP2C8|CYP1A1|CYP2D6|CYP2A6|CYP2C9||||||||||',
 'CYP|CYP2C19|CYP1A2|CYP2B6|CYP3A4|CYP1A1|CYP2C8|CYP2D6|CYP2C9|||||||||||',
 'UGT|CYP|AKR|UGT1A4|CYP3A4|UGT1A1|CYP2D6|AKR1C1|UGT1A3|AKR1C2||||||||||',
 'CES|CYP|UG

In [163]:
df=pd.read_csv("new_predict_add_true_drugbank_47_51.csv")

In [164]:
df

,predict_enzymes,predict_metabolites,substrate_smiles,true_label,true_enzymes,new_output
0,CYP1A2|CYP2A6|CYP2B6|CYP2C8|CYP2C9|CYP2C19|CYP...,CYP1A2|Cc1ncc2n1-c1cc(O)c(Cl)cc1C(c1ccccc1F)=N...,CC1=NC=C2N1C1=CC=C(Cl)C=C1C(C1=CC=CC=C1F)=NC2O,CYP3A4|CYP3A5|OCc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc...,CYP3A4|CYP3A5|UGT1A4|UGT2B4|UGT2B7,CYP3A4|OCc1ncc2n1-c1ccc(Cl)cc1C(c1ccccc1F)=NC2...
1,UGT|SULT,UGT|O=C(O)C1OC(O[C@H]2[C@H](c3ccc(Cl)c(Cc4ccc(...,OC[C@H]1O[C@@H](C2=CC=C(Cl)C(CC3=CC=C(OCCOC4CC...,UGT1A1|UGT1A9|O=C(O)[C@H]1OC(O[C@@H]2[C@@H](O)...,CYP2C8|CYP2D6|CYP3A4|UGT1A1|UGT1A9,UGT1A1|O=C(O)[C@H]1OC(O[C@@H]2[C@@H](O)[C@H](c...
2,UGT|SULT,UGT|COc1cc2c(cc1OC)[C@H]1C[C@@H](OC3OC(C(=O)O)...,COC1=CC2=C(C=C1OC)[C@H]1C[C@@H](O)[C@H](CC(C)C...,CYP2D6|COc1cc2c(cc1O)[C@H]1C[C@@H](O)[C@H](CC(...,CYP2D6,CYP2D6|COc1cc2c(cc1O)[C@H]1C[C@@H](O)[C@H](CC(...
3,UGT|SULT|EC 2.3.1.13|EC 2.3.1.7,UGT|COc1cc(C)c2[nH]ccc2c1CN1CC[C@H](OC2OC(C(=O...,COC1=CC(C)=C2NC=CC2=C1CN1CC[C@H](O)C[C@H]1C1=C...,UGT1A1|UGT1A3|UGT1A8|COc1cc(C)c2[nH]ccc2c1CN1C...,UGT1A1|UGT1A3|UGT1A8,UGT1A1|COc1cc(C)c2[nH]ccc2c1CN1CC[C@H](O)C[C@H...
4,AOX|CYP1A2|CYP2A6|CYP2B6|CYP2C8|CYP2C9|CYP2C19...,AOX|<separated>AOX|CC1(C)CC(=O)N(CCCCN2CCN(c3n...,CC1(C)CC(=O)N(CCCCN2CCN(C3=NC=CC=N3)CC2)C(=O)C1,CYP3A4|CC1(C)CC(=O)N(CCCCN2CCN(c3ncccn3)CC2)C(...,CYP3A4,CYP3A4|CC1(C)CC(=O)N(CCCCN2CCN(c3ncccn3)CC2)C(...
5,EC 4.2.1.84|EC 3.5.5.1,EC 4.2.1.84|CN1CC[C@@H](NC(=O)Nc2ccc(C(N)=O)cc...,CN1CC[C@@H](NC(=O)NC2=CC=C(C#N)C=C2)C[C@@H]1C1...,CYP3A4|CN1C=C[C@@H](NC(=O)Nc2ccc(C#N)cc2)C[C@@...,CYP3A4,CYP3A4|CN1C=C[C@@H](NC(=O)Nc2ccc(C#N)cc2)C[C@@...
6,EC 3.5.2.X|UGT,EC 3.5.2.X|CNc1ccc(Cl)cc1N(C(=O)CC(=O)O)c1ccc(...,CN1C(=O)CC(=O)N(C2=CC=C(O)C=C2)C2=CC(Cl)=CC=C12,CYP2B6|CYP2C9|CYP3A4|O=C1CC(=O)N(c2ccc(O)cc2)c...,CYP2B6|CYP2C9|CYP3A4,CYP2B6|O=C1CC(=O)N(c2ccc(O)cc2)c2cc(Cl)ccc2N1<...
7,ADH|UGT|SULT|CYP2B6|CYP2C8|CYP2C9|CYP2C19|CYP2...,ADH|Cc1nc(Nc2ncc(C(=O)Nc3c(C)cccc3Cl)s2)cc(N2C...,CC1=NC(NC2=NC=C(C(=O)NC3=C(C)C=CC=C3Cl)S2)=CC(...,CYP3A4|Cc1nc(Nc2ncc(C(=O)Nc3c(C)cccc3Cl)s2)cc(...,CYP3A4,CYP3A4|Cc1nc(Nc2ncc(C(=O)Nc3c(C)cccc3Cl)s2)cc(...
8,CYP1A2|CYP2E1|CYP2A6|CYP2C9|CYP2C19|CYP2D6|CYP...,CYP1A2|CCOc1ccc(S(=O)(=O)N2CCN(CC)CC2)cc1-c1nn...,CCCC1=NC(C)=C2C(=O)N=C(C3=CC(S(=O)(=O)N4CCN(CC...,CYP3A4|CCCc1nc(C)c2c(=O)nc(-c3cc(S(=O)(=O)N4CC...,CYP3A4,CYP3A4|CCCc1nc(C)c2c(=O)nc(-c3cc(S(=O)(=O)N4CC...
9,CYP1A2|CYP2A6|CYP2B6|AOX|CYP2C8|CYP2C9|CYP2C19...,CYP1A2|FC1=CC=C(C=C1)C2(OCC3=CC(C#N)=C(C=C32)O...,N#CC1=CC=C2C(=C1)COC2(CCC=O)C1=CC=C(F)C=C1,AOX|N#Cc1ccc2c(c1)COC2(CCC(=O)O)c1ccc(F)cc1,AOX,AOX|N#Cc1ccc2c(c1)COC2(CCC(=O)O)c1ccc(F)cc1


In [165]:
df['our_predict']=all

In [166]:

def process_predict(predict):
    # 1. 先用 '|' 拆分字符串
    parts = predict.split('|')
    
    # 2. 过滤掉空字符串
    non_empty_parts = [part for part in parts if part]
    
    # 3. 使用 '|' 重新连接非空部分
    return '|'.join(non_empty_parts)

# 对 df 的 predict 列进行处理
df['our_predict'] = df['our_predict'].apply(process_predict)




In [2]:
df=pd.read_csv("predict_with_enzymes.csv")

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("predict_with_enzymes.csv")

In [3]:
import pandas as pd
from rdkit import Chem
from rdkit.DataStructs import TanimotoSimilarity
from rdkit.Chem import AllChem

top=[3,5,7]
for top_i in top:
    corr = 0 
    at_least_one, at_least_half, all_metabolites = 0, 0, 0
    len_preds, len_corr_preds, len_actual_metabolites, invalid = 0, 0, 0, 0

    # 定义相似度函数

    # 遍历每一行
    for i in range(len(df)):
        # substrate = df.iloc[i]['substrate']
        true_metabolites = df.iloc[i]['true_enzymes'].strip().split('|')
        original_preds = df.iloc[i]['our_predict'].strip().split('|')[:top_i]
        # original_preds = df.iloc[i][f"{top}_new"].strip().split('|')
        
        unique_preds = list(set(original_preds))

        original_preds = unique_preds  # 去重后的预测值
        len_preds += len(original_preds)

        corr_preds = []
        # preds_to_remove = []
        
        for true_metabolite in true_metabolites:
            for pred in original_preds:
                if pred==true_metabolite:
                    corr_preds.append(pred)
                    # preds_to_remove.append(pred)
                    break  # 跳出内层循环，防止同一预测值多次计入

        # 移除已匹配的预测值，确保每个预测值只匹配一次
        # original_preds = [pred for pred in original_preds if pred not in preds_to_remove]
        
        if len(corr_preds) > 0:
            at_least_one += 1
        if len(corr_preds) >= len(true_metabolites) / 2:
            at_least_half += 1
        if len(corr_preds) == len(true_metabolites):
            all_metabolites += 1

        len_corr_preds += len(corr_preds)
        len_actual_metabolites += len(true_metabolites)
    precision = len_corr_preds / len_preds if len_preds > 0 else 0
    recall = len_corr_preds / len_actual_metabolites if len_actual_metabolites > 0 else 0
    
    # 计算 F1 score
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    with open("results_enzyme_unique.log", 'a') as f:
        f.write(
            ",".join([
                str("top"+str(top_i)), 
                "{:.2%}".format(at_least_one / len(df)), # at_least_one_metabolite
                "{:.2%}".format(at_least_half / len(df)), # at_least_half_metabolites
                "{:.2%}".format(all_metabolites / len(df)), # all_metabolites
                str(len_corr_preds),  # correct_predictions
                str(len_preds),  # total_predictions
                str(len_actual_metabolites),
                "{:.2%}".format(len_corr_preds / len_preds), # precision
                "{:.2%}".format(len_corr_preds / len_actual_metabolites), # recall
                "{:.2%}".format(f1_score)    # F1 score
            ]) + "\n"
        )



## if you have true labels, then can calculate the metrics

In [168]:
import pandas as pd
from rdkit import Chem
from rdkit.DataStructs import TanimotoSimilarity
from rdkit.Chem import AllChem

top=[5,10,15,20]
for top_i in top:
    corr = 0 
    at_least_one, at_least_half, all_metabolites = 0, 0, 0
    len_preds, len_corr_preds, len_actual_metabolites, invalid = 0, 0, 0, 0

    # 定义相似度函数

    # 遍历每一行
    for i in range(len(df)):
        # substrate = df.iloc[i]['substrate']
        true_metabolites = df.iloc[i]['true_enzymes'].strip().split('|')
        original_preds = df.iloc[i]['our_predict'].strip().split('|')[:top_i]
        # original_preds = df.iloc[i][f"{top}_new"].strip().split('|')
        
        unique_preds = list(set(original_preds))

        original_preds = unique_preds  # 去重后的预测值
        len_preds += len(original_preds)

        corr_preds = []
        # preds_to_remove = []
        
        for true_metabolite in true_metabolites:
            for pred in original_preds:
                if pred==true_metabolite:
                    corr_preds.append(pred)
                    # preds_to_remove.append(pred)
                    break  # 跳出内层循环，防止同一预测值多次计入

        # 移除已匹配的预测值，确保每个预测值只匹配一次
        # original_preds = [pred for pred in original_preds if pred not in preds_to_remove]
        
        if len(corr_preds) > 0:
            at_least_one += 1
        if len(corr_preds) >= len(true_metabolites) / 2:
            at_least_half += 1
        if len(corr_preds) == len(true_metabolites):
            all_metabolites += 1

        len_corr_preds += len(corr_preds)
        len_actual_metabolites += len(true_metabolites)
    precision = len_corr_preds / len_preds if len_preds > 0 else 0
    recall = len_corr_preds / len_actual_metabolites if len_actual_metabolites > 0 else 0
    
    # 计算 F1 score
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    with open("results_enzyme_unique.log", 'a') as f:
        f.write(
            ",".join([
                str("top"+str(top_i)), 
                "{:.2%}".format(at_least_one / len(df)), # at_least_one_metabolite
                "{:.2%}".format(at_least_half / len(df)), # at_least_half_metabolites
                "{:.2%}".format(all_metabolites / len(df)), # all_metabolites
                str(len_corr_preds),  # correct_predictions
                str(len_preds),  # total_predictions
                str(len_actual_metabolites),
                "{:.2%}".format(len_corr_preds / len_preds), # precision
                "{:.2%}".format(len_corr_preds / len_actual_metabolites), # recall
                "{:.2%}".format(f1_score)    # F1 score
            ]) + "\n"
        )



In [174]:
# 定义酶的大类前缀列表
enzyme_prefixes = ['CYP', 'UGT', 'SULT', 'AKR', 'GST', 'CES', 'FMO', 
                   'ALDH', 'NAT', 'COMT', 'AOX', 'XDH', 'EPHX', 'NQO', 'ADH']

# 函数：提取酶的大类前缀
def extract_enzyme_prefix(enzyme_name):
    for prefix in enzyme_prefixes:
        if enzyme_name.startswith(prefix):
            return prefix
    return None  # 如果不匹配任何前缀，返回 None

# 定义top值列表
top_values = [1, 2, 3, 4]
for top_i in top_values:
    # 初始化统计变量
    at_least_one, at_least_half, all_metabolites = 0, 0, 0
    len_preds, len_corr_preds, len_actual_metabolites = 0, 0, 0

    # 遍历每一行数据
    for i in range(len(df)):
        # 获取真实酶和预测酶
        true_metabolites = df.iloc[i]['true_enzymes'].strip().split('|')
        original_preds = str(df.iloc[i]['our_predict']).strip().split('|')[:top_i]

        # 提取真实酶和预测酶的大类前缀
        true_prefixes = list(set([extract_enzyme_prefix(enzyme) for enzyme in true_metabolites]))
        unique_preds = list(set(original_preds))
        pred_prefixes = list(set([extract_enzyme_prefix(pred) for pred in unique_preds if extract_enzyme_prefix(pred) is not None]))

        # 更新预测数量统计
        len_preds += len(pred_prefixes)

        # 统计匹配的预测前缀
        corr_preds = list(set([pred for pred in pred_prefixes if pred in true_prefixes]))

        # 更新匹配统计
        if len(corr_preds) > 0:
            at_least_one += 1
        if len(corr_preds) >= len(true_metabolites) / 2:
            at_least_half += 1
        if len(corr_preds) == len(true_metabolites):
            all_metabolites += 1

        len_corr_preds += len(corr_preds)
        len_actual_metabolites += len(true_metabolites)



    precision = len_corr_preds / len_preds if len_preds > 0 else 0
    recall = len_corr_preds / len_actual_metabolites if len_actual_metabolites > 0 else 0
    
    # 计算 F1 score
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    # 写入日志文件
    with open("results_enzyme_prefix_accuracy_new.log", 'a') as f:
        f.write(
            ",".join([
                f"top{top_i}", 
                "{:.2%}".format(at_least_one / len(df)),  # 至少一个匹配
                "{:.2%}".format(at_least_half / len(df)),  # 至少一半匹配
                "{:.2%}".format(all_metabolites / len(df)),  # 全部匹配
                str(len_corr_preds),  # 正确预测数
                str(len_preds),  # 总预测数
                str(len_actual_metabolites),  # 实际代谢物数
                # "{:.2%}".format(len_corr_preds / len_preds if len_preds > 0 else 0),  # 精度
                # "{:.2%}".format(len_corr_preds / len_actual_metabolites if len_actual_metabolites > 0 else 0) , # 召回率
                "{:.2%}".format(precision),  # 精度
                "{:.2%}".format(recall),     # 召回率
                "{:.2%}".format(f1_score)    # F1 score

            ]) + "\n"
        )


In [170]:
123

123

In [171]:
# # 大类前缀列表
# enzyme_prefixes = ['CYP', 'UGT', 'SULT', 'AKR', 'GST', 'CES', 'FMO', 
#                    'ALDH', 'NAT', 'COMT', 'AOX', 'XDH', 'EPHX', 'NQO', 'ADH']

# # 函数：提取酶的大类前缀
# def extract_enzyme_prefix(enzyme_name):
#     for prefix in enzyme_prefixes:
#         if enzyme_name.startswith(prefix):
#             return prefix
#     return None  # 如果不匹配任何前缀，返回 None

# top = [5, 10, 15, 20]
# for top_i in top:
#     corr = 0 
#     at_least_one, at_least_half, all_metabolites = 0, 0, 0
#     len_preds, len_corr_preds, len_actual_metabolites, invalid = 0, 0, 0, 0

#     # 遍历每一行
#     for i in range(len(df)):
#         # 获取真实酶和预测酶
#         true_metabolites = df.iloc[i]['true_enzymes'].strip().split('|')
#         original_preds = str(df.iloc[i]['our_predict']).strip().split('|')[:top_i]
        
#         # 提取真实酶的大类前缀
#         true_prefixes = [extract_enzyme_prefix(enzyme) for enzyme in true_metabolites]
        
#         # 去重后的预测酶前缀
#         unique_preds = list(set(original_preds))
#         pred_prefixes = [extract_enzyme_prefix(pred) for pred in unique_preds]

#         # 过滤掉无法识别前缀的预测值
#         pred_prefixes = [prefix for prefix in pred_prefixes if prefix is not None]

#         len_preds += len(pred_prefixes)

#         corr_preds = []
        
#         for true_prefix in true_prefixes:
#             for pred_prefix in pred_prefixes:
#                 if pred_prefix == true_prefix:
#                     corr_preds.append(pred_prefix)
#                     break  # 跳出内层循环，防止同一预测值多次计入

#         # 统计匹配结果
#         if len(corr_preds) > 0:
#             at_least_one += 1
#         if len(corr_preds) >= len(true_metabolites) / 2:
#             at_least_half += 1
#         if len(corr_preds) == len(true_metabolites):
#             all_metabolites += 1

#         len_corr_preds += len(corr_preds)
#         len_actual_metabolites += len(true_metabolites)

#     # 写入日志文件
#     with open("results_enzyme_prefix_accuracy.log", 'a') as f:
#         f.write(
#             ",".join([
#                 str("top"+str(top_i)), 
#                 "{:.2%}".format(at_least_one / len(df)), # at_least_one_metabolite
#                 "{:.2%}".format(at_least_half / len(df)), # at_least_half_metabolites
#                 "{:.2%}".format(all_metabolites / len(df)), # all_metabolites
#                 str(len_corr_preds),  # correct_predictions
#                 str(len_preds),  # total_predictions
#                 str(len_actual_metabolites),
#                 "{:.2%}".format(len_corr_preds / len_preds), # precision
#                 "{:.2%}".format(len_corr_preds / len_actual_metabolites), # recall
#             ]) + "\n"
#         )


In [173]:
df.to_csv("predict_with_enzymes.csv",index=False)